### Qwen2.5-7B-Instruct LLM run on Aristotle pipeline
### This is run in Runpod/remote Jupyter environment

In [ ]:
!apt update

In [ ]:
!apt install git-lfs

In [ ]:
!pip install transformers safetensors sentencepiece huggingface-hub accelerate bitsandbytes tqdm openai backoff retrying protobuf

In [ ]:
!git lfs install

In [ ]:
!git clone https://huggingface.co/Qwen/Qwen2.5-7B-Instruct LLM_MODELS/Qwen2.5-7B-Instruct

In [ ]:
import os
from pathlib import Path

os.environ["TOKENIZERS_PARALLELISM"] = "false"
######### Also useful to reduce thread contention:
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"

snapshot_path = "/workspace/LLM_MODELS/Qwen2.5-7B-Instruct" ############## <--- Change this based on platform and models

os.environ["LOCAL_MODEL_PATH"] = snapshot_path
os.environ["LLM_MODEL"] = snapshot_path

######## enable 4-bit for quant (and bitsandbytes is set up)
os.environ["LLM_LOAD_IN_4BIT"] = "1"  # or "0" to disable quantization

print("LOCAL_MODEL_PATH =", os.environ["LOCAL_MODEL_PATH"])
print("LLM_MODEL =", os.environ["LLM_MODEL"])

In [ ]:
#Test for Max Time generation stopping criteria
from llm_backends import HFBackend
hb = HFBackend(local_model_path=snapshot_path, quantize_4bit=True)
print("Calling short test (3s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=3.0)
print("Result length:", len(res))
print(res[:1000])

print("Calling short test (10s max_time)...")
res = hb.generate("Write a long list of words and sentences: ", max_new_tokens=1024, max_time=10.0)
print("Result length:", len(res))
print(res[:1000])

In [ ]:
#!python run_pipeline.py --dataset_name ProntoQA --sample_pct 0 --batch_size 1
#print("\nFinished translating dataset\n")

In [ ]:
os.environ["LLM_WORKER_MAX_TIME"] = "450"  # Change this every process (translate, decompose, search_resolve), different time limit is needed in seconds.

# Translation with original prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation --split dev --save_path results_translated/prompts_original --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# Translation with modified prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_modified --split dev --save_path results_translated/prompts_modified --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# Translation with refined prompts
#!python translate_to_fol.py --data_path manual_data_translated --dataset_name ProntoQA --sample_pct 100 --prompts_folder manual_prompts_translated --prompts_file translation_refine --split dev --save_path results_translated/prompts_refine --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
# Decomposition with refined prompts
!python decompose_to_cnf.py --data_path results_translated_translation/v3/prompts_refine --dataset_name ProntoQA --sample_pct 10 --prompts_folder manual_prompts_translated --prompts_file and_or_decomposer_refine --save_path results_translated_decomposition/prompts_refine --model_name $LLM_MODEL --batch_num 1 --max_new_tokens 6700

In [ ]:
#!python negate.py --dataset_name ProntoQA --save_path results_translated --model_name /workspace/LLM_MODELS/Qwen2.5-7B-Instruct

In [ ]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/Qwen2.5-7B-Instruct --batch_num 1 --negation False --max_new_tokens 8000

In [ ]:
#!python search_resolve.py --data_path manual_data_translate --dataset_name ProntoQA --prompts_folder manual_prompts_translated --split dev --save_path results_translated --model_name /workspace/LLM_MODELS/Qwen2.5-7B-Instruct --batch_num 1 --negation True --max_new_tokens 8000

In [ ]:
#!python evaluate.py --dataset_name ProntoQA --save_path resulys_translated --model_name /workspace/LLM_MODELS/Qwen2.5-7B-Instruct